In [1]:
import argparse
import pickle
import os

import numpy as np
import pandas as pd

In [2]:
def create_parser():
    parser = argparse.ArgumentParser(description="predict with DNN")
    parser.add_argument('--fingerprints', type=str, help='CSV with the fingerprints to predict', required=True)
    parser.add_argument('--dnn', type=str, help='pickled dnn', required=True)
    parser.add_argument('--preproc', type=str, help='pickled preproc', required=True)
    parser.add_argument('--save_to', type=str, help='save resulting dataframe to this CSV file', required=True)
    return parser

In [3]:
parser = create_parser()
args = parser.parse_args(['--fingerprints', 'data/CMM_vectorfingerprints.csv',
                         '--preproc', os.path.join('data', 'saved_models', 'v0', 'preprocessor.pkl'),
                         '--dnn', os.path.join('data', 'saved_models', 'v0', 'dnn.pkl'),
                         '--save_to', 'res1'])

In [7]:
fingerprints = pd.read_csv(args.fingerprints, dtype={'pid': str})
pid = fingerprints.pid.values
cmm_id = fingerprints.CMM_id.values
fingerprints = fingerprints[[col for col in fingerprints.columns if col not in ['pid', 'CMM_id']]].values
fingerprints = fingerprints.astype(np.float32)

In [8]:
with open(args.preproc, 'rb') as f:
    preprocessor = pickle.load(f)

/home/exouser/.local/lib/python3.9/site-packages/xgboost/compat.py:31: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/software/u22/anaconda/python3.9/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator VarianceThreshold from version 1.0.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [9]:
preprocessor.fgp_cols = np.arange(fingerprints.shape[1])

In [10]:
with open(args.dnn, 'rb') as f:
    dnn = pickle.load(f)

/software/u22/anaconda/python3.9/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator ColumnTransformer from version 1.0.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/software/u22/anaconda/python3.9/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator Pipeline from version 1.0.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/software/u22/anaconda/python3.9/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator QuantileTransformer from version 1.0.1 when using version 1.0.2. 

In [11]:
X_preprocessed = preprocessor.transform(fingerprints)

In [12]:
predictions = dnn.predict(X_preprocessed)

In [13]:
results = pd.DataFrame({
        'pid': pid,
        'cmm_id': cmm_id,
        'prediction': predictions
    })

In [14]:
results.head()

,pid,cmm_id,prediction
0,647,1,665.094727
1,1195,2,608.965210
2,5281976,3,585.646362
3,445995,4,795.100159
4,5280528,5,795.100159


In [21]:
results.to_csv('results/predicted_rt_db.csv')